---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.ensemble import RandomForestClassifier


def blight_model():
    # read csv file as DataFrame.
    train = pd.read_csv('train.csv', encoding='CP949')
    test = pd.read_csv('test.csv', encoding='CP949')
    
    # split data into features and label
    
    X = train.iloc[:, :-2]
    y = train.iloc[:, -1]
    total = pd.concat([X,test])
    # change agency name into agency number
    le = LabelEncoder().fit(total['agency_name'])
    X['agency_number'] = le.transform(X['agency_name'])
    test['agency_number'] = le.transform(test['agency_name'])
    
    # change inspector name into inspector number
    le = LabelEncoder().fit(total['inspector_name'])
    X['inspector_number'] = le.transform(X['inspector_name'])
    test['inspector_number'] = le.transform(test['inspector_name'])
    
    le = LabelEncoder().fit(total['country'])
    X['country_number'] = le.transform(X['country'])
    test['country_number'] = le.transform(test['country'])
    
    imp = Imputer(strategy='most_frequent')
    X['mailing_address_str_number'] = pd.to_numeric(X['mailing_address_str_number'], errors='coerce')
    X['impute_mailing_address_str_number'] = imp.fit_transform(X['mailing_address_str_number'].values.reshape(-1,1))
    test['mailing_address_str_number'] = pd.to_numeric(test['mailing_address_str_number'], errors='coerce')
    test['impute_mailing_address_str_number'] = imp.fit_transform(test['mailing_address_str_number'].values.reshape(-1,1))
    
    imp = Imputer(strategy='mean')
    X['impute_fine_amount'] = imp.fit_transform(X['fine_amount'].values.reshape(-1,1))
    test['impute_fine_amount'] = imp.fit_transform(test['fine_amount'].values.reshape(-1,1))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    target_feature = ['violation_street_number', 'impute_mailing_address_str_number', 'impute_fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'agency_number', 'inspector_number',
       'country_number']
    
    X_train_r = X_train[target_feature]
    X_test_r = X_test[target_feature]
    
    imp = Imputer(strategy='most_frequent')
    y_train_r = imp.fit_transform(y_train.values.reshape(-1,1))
    y_test = pd.to_numeric(y_test, errors='coerce')
    y_test_r = imp.transform(y_test.values.reshape(-1,1))
    
    rf = RandomForestClassifier().fit(X_train_r, y_train_r)
    
    target = test[target_feature]
    prediction = pd.DataFrame(rf.predict_proba(target))
    
    # Your code here
    
    return prediction[1]# Your answer here

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
blight_model()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2907: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0        0.00
1        0.00
2        0.00
3        0.00
4        0.30
5        0.00
6        0.10
7        0.70
8        0.00
9        0.00
10       0.00
11       0.00
12       0.00
13       0.10
14       0.10
15       0.10
16       0.00
17       0.00
18       0.00
19       0.00
20       0.00
21       0.00
22       0.00
23       0.20
24       0.30
25       0.00
26       0.10
27       0.00
28       0.00
29       0.00
         ... 
60971    0.00
60972    0.10
60973    0.30
60974    0.20
60975    0.00
60976    0.00
60977    0.20
60978    0.10
60979    0.44
60980    0.00
60981    0.20
60982    0.20
60983    0.30
60984    0.30
60985    0.20
60986    0.00
60987    0.10
60988    0.00
60989    0.40
60990    0.00
60991    0.10
60992    0.10
60993    0.10
60994    0.10
60995    0.00
60996    0.10
60997    0.10
60998    0.00
60999    0.00
61000    0.80
Name: 1, Length: 61001, dtype: float64